# Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import joblib
from keras.models import load_model
from datetime import datetime,date
from dateutil.relativedelta import relativedelta

# Reading and preprocessing of dataset

In [2]:
today_date = date.today().strftime("%Y-%m-%d")

In [3]:
dataset = pd.read_csv(f"D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\stock_prices_dataset\\Stocks_train_{today_date}.csv")

In [4]:
dataset_test = dataset.pivot(index='Date',columns='Stock',values='Close').reset_index()

In [5]:
dataset_test

Stock,Date,ASHOKLEY,CANBK,LICI,ONGC,SBIN
0,2022-06-06,138.35,210.30,777.40,154.00,463.55
1,2022-06-07,139.60,209.10,752.90,161.85,463.35
2,2022-06-08,138.50,210.95,738.00,163.35,471.05
3,2022-06-09,137.85,207.45,721.95,165.60,466.85
4,2022-06-10,137.80,208.05,709.70,164.60,461.80
...,...,...,...,...,...,...
732,2025-05-29,240.65,110.81,944.40,243.35,797.30
733,2025-05-30,236.00,114.82,953.55,239.45,812.40
734,2025-06-02,236.30,116.85,962.10,238.30,813.45
735,2025-06-03,236.05,115.85,953.80,237.20,809.75


In [6]:
dataset_test = dataset_test[dataset_test['Date']<today_date]

In [7]:
# Feature Scaling
num_cols = ['ASHOKLEY','CANBK','LICI','ONGC','SBIN']
sc = joblib.load('D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\Stock_predictor_BSE_2025\\model_files\\min_max_scaler.joblib')
test_data = sc.transform(dataset_test[num_cols])

In [8]:
# Creating a data structure with 60 time steps and 1 output
X_test = test_data[len(test_data)-60:len(test_data),:]

In [9]:
X_test = X_test.reshape(1,60,-1)

# Load the model and predict

In [10]:
regressor_load = load_model('D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\Stock_predictor_BSE_2025\\model_files\\stock_predictor.keras')

In [11]:
today_preds = regressor_load.predict(X_test)
today_preds = sc.inverse_transform(today_preds)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [12]:
today_preds_pd = pd.DataFrame(today_preds,columns=num_cols)
today_preds_pd['Date'] = today_date
today_preds_pd = today_preds_pd[['Date']+num_cols]

In [13]:
today_preds_pd

,Date,ASHOKLEY,CANBK,LICI,ONGC,SBIN
0,2025-06-04,229.864838,114.790794,941.679077,251.472214,808.377441


In [16]:
today_preds_pd.to_csv(f"D:\\Projects\\Jupyter_Lab\\Stock_market_predictor\\stock_prices_predictions\\Predictions_{today_date}.csv",index=False)